In [1]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 74.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 28.9 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9297 sha256=5687d6777e1e8a7721022ab0145f0502ad76f1de180bfec95d09a1eac180a97e
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

# **Load Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hazm
import string

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import tensorflow as tf


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SpatialDropout1D, Embedding, Bidirectional, LSTM, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight

from sklearn.metrics import classification_report
from collections import Counter
from sklearn.metrics._plot.confusion_matrix import confusion_matrix

# **Load Data and EDA**

In [2]:
data = pd.read_csv('Snappfood - Sentiment Analysis.csv', delimiter='\t', on_bad_lines='skip')
data

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0
...,...,...,...,...
69995,NaN,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1.0
69996,NaN,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1.0
69997,NaN,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0.0
69998,NaN,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  520 non-null    object 
 1   comment     70000 non-null  object 
 2   label       70000 non-null  object 
 3   label_id    69480 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.1+ MB


In [4]:
data.label_id.value_counts()

0.0    34916
1.0    34564
Name: label_id, dtype: int64

In [5]:
data.isnull().sum()

Unnamed: 0    69480
comment           0
label             0
label_id        520
dtype: int64

In [6]:
data = data[['comment', 'label_id']]
data

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0
...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,1.0
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,1.0
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,0.0
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,1.0


In [7]:
data.dropna(inplace=True)

<ipython-input-7-f1116dacf2bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [8]:
data

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0
...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,1.0
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,1.0
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,0.0
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,1.0


# **PreProcessing**

In [9]:
normalizer = hazm.Normalizer()
stemmer = hazm.Stemmer()
stopwords = hazm.stopwords_list()
punctuations = string.punctuation + "٬" + "،"

def clean_text(text):
    translator = str.maketrans('', '', punctuations)
    normalized_text = normalizer.normalize(text)
    cleaned_text = [stemmer.stem(word.translate(translator)) for word in hazm.word_tokenize(
        normalized_text) if word not in stopwords]

    return ' '.join(cleaned_text)

In [10]:
data.comment.apply(clean_text)

0                   واقعا حیف وق بنویس سرویس دهیتون افتضاح
1        قرار ۱ ساعته برسه ن ساع زود موقع  ببین چقدر پل...
2        قیم مدل اصلا کیفیت سازگار نداره  ظاهر فریبنده ...
3        درس اندازه کیف  امیداور کیفیتتون باشه مشتر همی...
4                                         شیرین وانیل مدل 
                               ...                        
69995    سلا فاکتور غذا سفار مید احتیاج دار موضوع قسم س...
69996                           سایز پیتزا سفارشات قبلا گذ
69997    قارچ اضافه اضافه کردهبود بود اضافه نمی‌کرد نمی...
69998                همرو ۲ ساع تاخیر اشتباه آوردن پول هوا
69999                                            فلفل تند 
Name: comment, Length: 69480, dtype: object

In [11]:
data

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0
...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,1.0
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,1.0
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,0.0
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,1.0


In [12]:
data.label_id.value_counts()

0.0    34916
1.0    34564
Name: label_id, dtype: int64

In [13]:
data['words_count'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))
max_len = data["words_count"].max()
max_len

<ipython-input-13-ad8e58f6f16c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_count'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))


378

In [14]:
texts = ' '.join(data['comment'])
tokens = hazm.word_tokenize(texts)
counter = Counter(tokens)
min_freg = 35
filtered = [word for word, count in counter.items() if count >= min_freg]
unique_words = set(filtered)
n_words = len(unique_words)
n_words

2527

# **Tokenization Pad Sequences**

In [16]:
X = data['comment']
Y = data['label_id']
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.1, random_state=4234)

In [18]:
tokenizer = Tokenizer(num_words = n_words)
tokenizer.fit_on_texts(xtrain)

def Tokenization_padSequences(x, maxlen = max_len):
    xseq = tokenizer.texts_to_sequences(x)
    xpad = pad_sequences(xseq, padding='post', maxlen = max_len)
    return xpad

xtrain_pad = Tokenization_padSequences(xtrain)
xtest_pad = Tokenization_padSequences(xtest)

In [19]:
xtest_pad

array([[ 145,   33, 1907, ...,    0,    0,    0],
       [  63,  122,   77, ...,    0,    0,    0],
       [ 444, 2160,    5, ...,    0,    0,    0],
       ...,
       [  11,   83,   17, ...,    0,    0,    0],
       [  97, 2055,   41, ...,    0,    0,    0],
       [   5,   29,   14, ...,    0,    0,    0]], dtype=int32)

In [21]:
sequences = tokenizer.texts_to_sequences(data['comment'])

print(data['comment'][10])
print(sequences[10])

اکلر فوق العاده بود اما بافت چیزکیک مونده بود و دوست نداشتم.
[125, 139, 1, 44, 2127, 1494, 95, 1, 2, 251, 292]


#**NetWork**

In [22]:
Metrics = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name = 'precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.F1Score(name='f1-score')
]

In [23]:
from sklearn.utils.class_weight import compute_class_weight
weights = compute_class_weight(class_weight='balanced', classes=np.unique(ytrain), y=ytrain)
weights = {0 : 0.99469285, 1: 1.00536409}
weights

{0: 0.99469285, 1: 1.00536409}

In [28]:
model = Sequential()
model.add(Embedding(n_words, 64, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(256, dropout=0.2)))
model.add(Dense(1, activation ='sigmoid'))
model.compile(optimizer='Adam', metrics= Metrics, loss = 'binary_crossentropy')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 378, 64)           161728    
                                                                 
 spatial_dropout1d_2 (Spati  (None, 378, 64)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_2 (Bidirecti  (None, 512)               657408    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 819649 (3.13 MB)
Trainable params: 819649 (3.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
np.unique(ytrain)

array([0., 1.])

In [29]:
model.fit(xtrain_pad, ytrain, epochs=10, batch_size=100, validation_data = (xtest_pad, ytest), class_weight=weights)

Epoch 1/10
626/626 [==============================] - 94s 135ms/step - loss: 0.3857 - accuracy: 0.8282 - precision: 0.7953 - recall: 0.8816 - f1-score: 0.6644 - val_loss: 0.3411 - val_accuracy: 0.8564 - val_precision: 0.8301 - val_recall: 0.8944 - val_f1-score: 0.6646
Epoch 2/10
626/626 [==============================] - 56s 89ms/step - loss: 0.3310 - accuracy: 0.8598 - precision: 0.8307 - recall: 0.9021 - f1-score: 0.6644 - val_loss: 0.3315 - val_accuracy: 0.8588 - val_precision: 0.8186 - val_recall: 0.9202 - val_f1-score: 0.6646
Epoch 3/10
626/626 [==============================] - 56s 90ms/step - loss: 0.3170 - accuracy: 0.8655 - precision: 0.8355 - recall: 0.9085 - f1-score: 0.6644 - val_loss: 0.3326 - val_accuracy: 0.8598 - val_precision: 0.8463 - val_recall: 0.8777 - val_f1-score: 0.6646
Epoch 4/10
626/626 [==============================] - 55s 87ms/step - loss: 0.3063 - accuracy: 0.8704 - precision: 0.8403 - recall: 0.9130 - f1-score: 0.6644 - val_loss: 0.3243 - val_accuracy: 0.

In [30]:
ypred_train = (model.predict(xtrain_pad) > 0.5).astype(int)
print(classification_report(ytrain, ypred_train))

1955/1955 [==============================] - 29s 14ms/step
              precision    recall  f1-score   support

         0.0       0.92      0.89      0.91     31426
         1.0       0.89      0.92      0.91     31106

    accuracy                           0.91     62532
   macro avg       0.91      0.91      0.91     62532
weighted avg       0.91      0.91      0.91     62532



In [31]:
ypred = (model.predict(xtest_pad) > 0.5).astype(int)
print(classification_report(ytest, ypred))

218/218 [==============================] - 3s 15ms/step
              precision    recall  f1-score   support

         0.0       0.86      0.85      0.86      3490
         1.0       0.85      0.86      0.86      3458

    accuracy                           0.86      6948
   macro avg       0.86      0.86      0.86      6948
weighted avg       0.86      0.86      0.86      6948



In [45]:
print(confusion_matrix(ypred, ytest))

[[2971  478]
 [ 519 2980]]


In [39]:
print(confusion_matrix(ypred_train, ytrain))

[[27981  2364]
 [ 3445 28742]]
